In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

In [3]:
baseball = pd.read_csv('best_model/engineered_data.csv')

In [4]:
X = baseball.drop(['Salary'], axis = 1)
y = baseball['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 621)

In [5]:
cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF']]

cat_transformer = Pipeline(
    steps = [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

num_transformer = Pipeline(
    steps = [
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ], remainder = 'passthrough'
)

X_transform = preprocessor.fit_transform(X)

selected_features = np.concatenate([
    np.array(num_columns),
    np.array(preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(cat_columns)),
    np.array(['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF'])
])


In [7]:
from keras.models import Sequential
from keras.layers import Dropout, Dense
from bayes_opt import BayesianOptimization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import warnings
import joblib

warnings.filterwarnings('ignore')


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size = .2, random_state = 621)

In [9]:
def dnn_model_score(neurons, dropout_rate, learning_rate, epochs, batch_size, patience, num_layers, **layer_neurons):
    model = Sequential()
    model.add(Dense(int(neurons), activation='relu', input_shape = (X_train.shape[1],)))
    model.add(Dropout(dropout_rate))

    for i in range(1, int(num_layers) + 1):
        model.add(Dense(int(layer_neurons[f'layer_neurons_{i}']), activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation = 'linear'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error', optimizer = optimizer)

    es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

    model.fit(X_train, y_train, validation_split = .2, epochs = int(epochs), batch_size = int(batch_size), callbacks = es, verbose = 0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return -mse

pbounds = {'neurons': (32, 256),
           'dropout_rate': (0.0, 0.5),
           'learning_rate': (0.01, 1),
           'epochs' : (100, 500),
           'batch_size' : (32, 500),
           'patience' : (20, 50),
           'num_layers': (1, 5)}

for i in range(1, 6):
    pbounds[f'layer_neurons_{i}'] = (32, 256)

optimizer = BayesianOptimization(f = dnn_model_score, pbounds = pbounds, random_state = 42)

optimizer.maximize(init_points = 5, n_iter = 10)

best_params = optimizer.max['params']
print("Best Hyperparameters:", best_params)


|   iter    |  target   | batch_... | dropou... |  epochs   | layer_... | layer_... | layer_... | layer_... | layer_... | learni... |  neurons  | num_la... | patience  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
| 1         | -2.276e+1 | 207.3     | 0.4754    | 392.8     | 166.1     | 66.95     | 66.94     | 45.01     | 226.0     | 0.6051    | 190.6     | 1.082     | 49.1      |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
| 2         | -2.398e+1 | 421.6     | 0.1062    | 172.7     | 73.08     | 100.2     | 149.5     | 128.8     | 97.24     | 0.6157    | 63.25     | 2.169     | 30.99     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
| 3         | -2.405e+1 | 245.4     | 0.3926    | 179.9     | 147.2     | 164.7     | 42.4      | 168.1     | 70.2      | 0.0744    | 244.6     | 4.863     | 44.25     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms

In [10]:
batch_size = int(best_params['batch_size'])
dropout_rate = best_params['dropout_rate']
epochs = int(best_params['epochs'])
neurons = []
neurons.append(int(best_params['neurons']))
neurons.append(int(best_params['layer_neurons_1']))
neurons.append(int(best_params['layer_neurons_2']))
neurons.append(int(best_params['layer_neurons_3']))
neurons.append(int(best_params['layer_neurons_4']))
neurons.append(int(best_params['layer_neurons_5']))
learning_rate = best_params['learning_rate']
num_layers = int(best_params['num_layers'])
patience = int(best_params['patience'])

In [11]:
model = Sequential()

model.add(Dense(name = 'Dense1', units = neurons[0], input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dropout(name = 'Dropout1', rate = dropout_rate))

for i in range(1, num_layers + 1):
    model.add(Dense(name = f'Dense{i + 1}', units = neurons[i], activation = 'relu'))
    model.add(Dropout(name = f'Dropout{i + 1}', rate = dropout_rate))

model.add(Dense(name = 'Output', units = 1, activation = 'linear'))

optimizer = Adam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, loss = 'mean_squared_error')

model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense1 (Dense)                  │ (None, 250)            │        19,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout1 (Dropout)              │ (None, 250)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2 (Dense)                  │ (None, 154)            │        38,654 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout2 (Dropout)              │ (None, 154)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │           155 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,559 (228.75 KB)

 Trainable params: 58,559 (228.75 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

history = model.fit(X_train, y_train, validation_split = .2, batch_size = batch_size, epochs = epochs, callbacks = es, verbose = 2)

Epoch 1/262
13/13 - 3s - 199ms/step - loss: 63482414235648.0000 - val_loss: 42525863706624.0000
Epoch 2/262
13/13 - 0s - 18ms/step - loss: 39605755707392.0000 - val_loss: 30419678396416.0000
Epoch 3/262
13/13 - 0s - 19ms/step - loss: 33617648549888.0000 - val_loss: 29283718266880.0000
Epoch 4/262
13/13 - 0s - 18ms/step - loss: 31119057616896.0000 - val_loss: 27809168752640.0000
Epoch 5/262
13/13 - 0s - 18ms/step - loss: 29824003342336.0000 - val_loss: 27760309305344.0000
Epoch 6/262
13/13 - 0s - 20ms/step - loss: 28683423186944.0000 - val_loss: 26364637872128.0000
Epoch 7/262
13/13 - 0s - 20ms/step - loss: 28391474462720.0000 - val_loss: 25859113091072.0000
Epoch 8/262
13/13 - 0s - 19ms/step - loss: 28075643371520.0000 - val_loss: 25343484231680.0000
Epoch 9/262
13/13 - 0s - 22ms/step - loss: 27096418091008.0000 - val_loss: 25264018948096.0000
Epoch 10/262
13/13 - 0s - 18ms/step - loss: 26878295408640.0000 - val_loss: 24960674299904.0000
Epoch 11/262
13/13 - 0s - 22ms/step - loss: 2642

In [13]:
test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [14]:
train_rmse = mean_squared_error(y_train, train_preds)
test_rmse = mean_squared_error(y_test, test_preds)

print('Deep Neural Network Metrics:')
print(f'Train RMSE: {np.sqrt(train_rmse)}')
print(f'Test RMSE: {np.sqrt(test_rmse)}')

Deep Neural Network Metrics:
Train RMSE: 3015039.9241755
Test RMSE: 4843637.290906582


In [ ]:
# save the model, pca, and preprocessor so that new data can be fit using the same criteria
#model.save('best_model/best_model.keras')
#joblib.dump(pca, 'dnn/pca45.joblib')
#joblib.dump(preprocessor, 'best_model/best_preprocssor.joblib')
